<a href="https://colab.research.google.com/github/ryderwishart/biblical-machine-learning/blob/main/topic_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyLDAvis
!pip install gensim==4.3.0
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Imports

In [2]:
from gensim import corpora
from gensim.models import CoherenceModel, LdaModel
from gensim.models import EnsembleLda
from gensim.corpora import Dictionary
from collections import Counter
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import os
import codecs
import pandas as pd

## Build Corpus

In [3]:
if 'macula-greek.tsv' not in [path for path in os.listdir()]:
    !wget -q "https://raw.githubusercontent.com/Clear-Bible/macula-greek/main/Nestle1904/TSV/macula-greek.tsv"

In [4]:
os.listdir()

['.config', 'macula-greek.tsv.1', 'macula-greek.tsv', 'sample_data']

In [5]:
# Import Macula Greek data
mg = pd.read_csv('macula-greek.tsv', sep='\t', header=0)

# Extract book, chapter, and verse into separate columns
mg[['book', 'chapter', 'verse']] = mg['ref'].str.extract(r'(\d?[A-Z]+)\s(\d+):(\d+)')

# Add columns for book + chapter, and book + chapter + verse for easier grouping
mg['book_chapter'] = mg['book'] + ' ' + mg['chapter'].astype(str)
mg['book_chapter_verse'] = mg['book_chapter'] + ':' + mg['verse'].astype(str)

# Display the updated data frame
mg.head()

,xml:id,ref,role,class,type,gloss,text,after,lemma,normalized,...,domain,ln,frame,subjref,referent,book,chapter,verse,book_chapter,book_chapter_verse
0,n40001001001,MAT 1:1!1,NaN,noun,common,[The] book,Βίβλος,,βίβλος,Βίβλος,...,033005,33.38,NaN,NaN,NaN,MAT,1,1,MAT 1,MAT 1:1
1,n40001001002,MAT 1:1!2,NaN,noun,common,of [the] genealogy,γενέσεως,,γένεσις,γενέσεως,...,010002 033003,10.24 33.19,NaN,NaN,NaN,MAT,1,1,MAT 1,MAT 1:1
2,n40001001003,MAT 1:1!3,NaN,noun,proper,of Jesus,Ἰησοῦ,,Ἰησοῦς,Ἰησοῦ,...,093001,93.169a,NaN,NaN,NaN,MAT,1,1,MAT 1,MAT 1:1
3,n40001001004,MAT 1:1!4,NaN,noun,proper,Christ,Χριστοῦ,,Χριστός,Χριστοῦ,...,093001,93.387,NaN,NaN,NaN,MAT,1,1,MAT 1,MAT 1:1
4,n40001001005,MAT 1:1!5,NaN,noun,common,son,υἱοῦ,,υἱός,υἱοῦ,...,010002,10.30,NaN,NaN,NaN,MAT,1,1,MAT 1,MAT 1:1


Let's filter out stopwords using a list from [Perseus](https://wiki.digitalclassicist.org/Stopwords_for_Greek_and_Latin).

In [6]:
perseus_stopwords = "μή, ἑαυτοῦ, ἄν, ἀλλ', ἀλλά, ἄλλος, ἀπό, ἄρα, αὐτός, δ', δέ, δή, διά, δαί, δαίς, ἔτι, ἐγώ, ἐκ, ἐμός, ἐν, ἐπί, εἰ, εἰμί, εἴμι, εἰς, γάρ, γε, γα, ἡ, ἤ, καί, κατά, μέν, μετά, μή, ὁ, ὅδε, ὅς, ὅστις, ὅτι, οὕτως, οὗτος, οὔτε, οὖν, οὐδείς, οἱ, οὐ, οὐδέ, οὐκ, περί, πρός, σύ, σύν, τά, τε, τήν, τῆς, τῇ, τι, τί, τις, τίς, τό, τοί, τοιοῦτος, τόν, τούς, τοῦ, τῶν, τῷ, ὑμός, ὑπέρ, ὑπό, ὡς, ὦ, ὥστε, ἐάν, παρά, σός".split(', ')
perseus_stopwords += "συ".split(' ')
# filter the DataFrame to exclude rows with class values in the exclude_list
filtered_mg = mg[~mg['normalized'].isin(perseus_stopwords)]
difference = len(mg) - len(filtered_mg)
print(f'{difference} rows removed using stopwords')
# Also exclude certain parts of speech
pos_exclude_list = ['det', 'prep', 'pron', 'conj', 'ptcl']
cur_len = len(filtered_mg)
filtered_mg = filtered_mg[~filtered_mg['class'].isin(pos_exclude_list)]
print(f'{cur_len - len(filtered_mg)} more rows removed using parts of speech')

48538 rows removed using stopwords
20951 more rows removed using parts of speech


Create an ensemble topic model for each book of the New Testament, using lemmas and ignoring stopwords.

In [7]:
book_lemmas = {}  # Initialize a dictionary to hold the corpora for each book

def get_lemmas(group):
    return group['lemma'].values.tolist()

all_lemmas = []

for book in filtered_mg['book'].unique():  # Loop over unique books in the dataframe
    print(f'Processing {book}...')
    book_df = filtered_mg[filtered_mg['book'] == book]  # Filter the dataframe for the current book
    corpus = []  # Initialize an empty list to hold the tokens for the current book
    grouped = book_df.groupby('book_chapter').apply(get_lemmas)
    # Loop over rows in the filtered dataframe
    for chapter in grouped:  
        corpus += [chapter]  # Add the tokens to the corpus for the current book
        all_lemmas += [chapter]
    book_lemmas[book] = corpus  # Add the corpus for the current book to the dictionary

Processing MAT...
Processing MRK...
Processing LUK...
Processing JHN...
Processing ACT...
Processing ROM...
Processing 1CO...
Processing 2CO...
Processing GAL...
Processing EPH...
Processing PHP...
Processing COL...
Processing 1TH...
Processing 2TH...
Processing 1TI...
Processing 2TI...
Processing TIT...
Processing PHM...
Processing HEB...
Processing JAS...
Processing 1PE...
Processing 2PE...
Processing 1JN...
Processing 2JN...
Processing 3JN...
Processing JUD...
Processing REV...


In [8]:
# Create a dictionary for all words in each corpus
dictionary = corpora.Dictionary(all_lemmas)
dictionary

In [9]:
models = dict()

for book, lemmas in book_lemmas.items():
    models[book] = {
        'book': book, 
        'corpus': [dictionary.doc2bow(chapter) for chapter in lemmas], 
        'model': None 
    }

## Train LDA Model

In [10]:
ensemble_workers = 4
num_models = 3
distance_workers = 4 # Note from Gensim: After training all the models, some distance computations are required which can take quite some time as well. You can speed this up by using workers for that as well.
num_topics = 100
passes = 20

In [11]:
for book, data in models.items():
    print(f'Generating ensemble models for {book}...')
    ensemble = EnsembleLda(
        corpus=data['corpus'],
        id2word=dictionary,
        num_topics=num_topics,
        passes=passes,
        num_models=num_models,
        topic_model_class=LdaModel,
        ensemble_workers=ensemble_workers,
        distance_workers=distance_workers
    )
    data['ensemble'] = ensemble
    data['model'] = ensemble.generate_gensim_representation()
    break

Generating ensemble models for MAT...


In [12]:
vis_model = models['MAT']['model']
vis_corpus = models['MAT']['corpus']

Since we trained an ensemble model, we need to use the `generate_gensim_representation()` method to return an `LdaModel` class as the visualization library is expecting.

## Evaluate Model

In [13]:
# Compute the coherence score
coherence_model_lda = CoherenceModel(
    model=vis_model,
    corpus=vis_corpus,
    dictionary=dictionary,
    coherence='u_mass'
)
coherence_lda = coherence_model_lda.get_coherence()
print(f'Coherence Score: {coherence_lda:.3f}')

Coherence Score: -0.711


## Visualize Topics

In [17]:
# Visualize the topics using pyLDAvis
vis = gensimvis.prepare(vis_model, vis_corpus, dictionary)
pyLDAvis.display(vis)

/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


## Tuning the model ([source](https://radimrehurek.com/gensim/auto_examples/tutorials/run_ensemblelda.html#tuning))

Different from `LdaModel`, the number of resulting topics varies greatly depending on the clustering parameters.

You can provide those in the `recluster()` function or the `EnsembleLda` constructor.

Play around until you get as many topics as you desire, which however may reduce their quality. If your ensemble doesn't have enough topics to begin with, you should make sure to make it large enough.

Having an epsilon that is smaller than the smallest distance doesn't make sense. Make sure to chose one that is within the range of values in `asymmetric_distance_matrix`.

In [15]:
import numpy as np
shape = ensemble.asymmetric_distance_matrix.shape
without_diagonal = ensemble.asymmetric_distance_matrix[~np.eye(shape[0], dtype=bool)].reshape(shape[0], -1)
print(without_diagonal.min(), without_diagonal.mean(), without_diagonal.max())

ensemble.recluster(eps=0.09, min_samples=2, min_cores=2)

print(len(ensemble.get_topics()))

0.0 0.24615761657260232 0.995612651369295
3
